In [6]:
import pandas as pd
import pathlib
import multiprocessing
import time
import importlib
from scripts.stat_utils import collect_study_stats, collect_databank_stats
from scripts.io_utils import walk

In [7]:
# Define study metadata directory
studies_metadata_dir = pathlib.Path("../data/metadata")

# Collect metadata file paths
metadata_files = list(walk(studies_metadata_dir))

# Calculate statistics for each image attribute within each study
all_results_list = list()
for metadata_path in metadata_files:
    collect_study_stats(metadata_path, all_results_list)

stat_results_df = pd.DataFrame(
    data=all_results_list,
    columns=["Study_Name", "Attribute", "S", "H", "NME", "J", "E", "GC"],
)

# Make directories
stats_dir = pathlib.Path("../data/statistics")
pathlib.Path.mkdir(stats_dir, exist_ok=True)

# Save individual stats as parquet file
output_file = pathlib.Path(stats_dir, f"individual_studies_diversity.parquet.gzip")
stat_results_df.to_parquet(output_file, compression="gzip")

/home/parkerhicks/anaconda3/envs/microscopy-data/lib/python3.10/site-packages/skbio/diversity/alpha/_base.py:793: RuntimeWarning: invalid value encountered in double_scalars
  return shannon(counts, base=np.e) / np.log(observed_otus(counts))
/home/parkerhicks/anaconda3/envs/microscopy-data/lib/python3.10/site-packages/skbio/diversity/alpha/_base.py:793: RuntimeWarning: invalid value encountered in double_scalars
  return shannon(counts, base=np.e) / np.log(observed_otus(counts))
/home/parkerhicks/anaconda3/envs/microscopy-data/lib/python3.10/site-packages/skbio/diversity/alpha/_base.py:793: RuntimeWarning: invalid value encountered in double_scalars
  return shannon(counts, base=np.e) / np.log(observed_otus(counts))


In [8]:
# Collect databank stats
databank_stats = collect_databank_stats(metadata_dir=studies_metadata_dir)

# Save databank stats as parquet file
output_file = pathlib.Path(stats_dir, f"databank_diversity.parquet.gzip")
databank_stats.to_parquet(output_file, compression="gzip")

/home/parkerhicks/anaconda3/envs/microscopy-data/lib/python3.10/site-packages/skbio/diversity/alpha/_base.py:793: RuntimeWarning: invalid value encountered in double_scalars
  return shannon(counts, base=np.e) / np.log(observed_otus(counts))


In [9]:
study_stats = pd.read_parquet(f"{stats_dir}/individual_studies_diversity.parquet.gzip")
print(study_stats.head())

databank_stats = pd.read_parquet(f"{stats_dir}/databank_diversity.parquet.gzip")
print(databank_stats.head())

                              Study_Name       Attribute     S          H  \
0  idr0080-way-perturbation_screenA_2701         well_id  6912  12.754888   
1  idr0080-way-perturbation_screenA_2701  imaging_method     1  -0.000000   
2  idr0080-way-perturbation_screenA_2701        organism     1  -0.000000   
3  idr0080-way-perturbation_screenA_2701   organism_part     1  -0.000000   
4  idr0080-way-perturbation_screenA_2701       cell_line     3   1.584963   

     J        GC  
0  1.0 -0.000145  
1  NaN -1.000000  
2  NaN -1.000000  
3  NaN -1.000000  
4  1.0 -0.333333  
    Attribute      S          H         J        GC
0   screen_id      3   1.241427  0.783253  0.030508
1  study_name      3   1.241427  0.783253  0.030508
2    plate_id    334   7.926605  0.945478  0.368406
3  plate_name    334   7.926605  0.945478  0.368406
4     well_id  28320  14.789534  1.000000 -0.000035
